In [150]:
import pandas as pd
import features_processing_utils as fpu
import hopsworks
import numpy as np

Log into Hopsworks

In [151]:
project = hopsworks.login()
# util.purge_project(proj)

2025-01-07 16:40:14,236 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 16:40:14,250 INFO: Initializing external client
2025-01-07 16:40:14,251 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 16:40:16,096 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1149079


In [152]:
fs = project.get_feature_store()

In [153]:
gossipcop_fake = pd.read_csv("FakeNewsNet/gossipcop_fake.csv")

In [78]:
gossipcop_new_features = fpu.complete_processing(gossipcop_fake, 1, "gossipcop_fake_posts.csv", "gossipcop_fake_features.csv", 2000, 3000)

 14%|█▍        | 144/1000 [09:35<14:15,  1.00it/s]  

In [ ]:
gossipcop_new_features.info()

In [79]:
gossipcop_real = pd.read_csv("FakeNewsNet/gossipcop_real.csv")

In [80]:
#gossicop_fake_features = fpu.complete_processing(gossipcop_fake, 1, "gossipcop_fake_posts_4.csv", "gossipcop_fake_features_4.csv", 3000, 4000)
# gossicop_fake_features.info()

In [81]:
# fpu.complete_processing(gossipcop_fake, 1, "gossipcop_fake_posts_5.csv", "gossipcop_fake_features_5.csv", 4000, 5000)

In [82]:
# fpu.complete_processing(gossipcop_real, 0, "gossipcop_real_posts_3.csv", "gossipcop_real_features_3.csv", 2000, 3000)

In [83]:
# fpu.complete_processing(gossipcop_real, 0, "gossipcop_real_posts_4.csv", "gossipcop_real_features_4.csv", 3000, 4000)

In [84]:
# gossicop_real_features = fpu.complete_processing(gossipcop_real, 0, "gossipcop_real_posts_5.csv", "gossipcop_real_features_5.csv", 4000)
# gossicop_real_features.info()

In [149]:
gossipcop_features = pd.read_csv("data/gossipcop_features.csv")
gossipcop_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   news_id                    328 non-null    object 
 1   average_followers          328 non-null    float64
 2   average_follows            328 non-null    float64
 3   repost_total               328 non-null    int64  
 4   post_total                 328 non-null    int64  
 5   repost_percentage          328 non-null    float64
 6   label                      328 non-null    int64  
 7   average_repost             328 non-null    float64
 8   average_favorite           328 non-null    float64
 9   news_lifetime              328 non-null    float64
 10  nb_users_10_hours          328 non-null    int64  
 11  average_time_difference    328 non-null    float64
 12  retweet_percentage_1_hour  328 non-null    float64
dtypes: float64(8), int64(4), object(1)
memory usage: 3

In [135]:
# change names of columns to add underscore
gossipcop_features.columns = gossipcop_features.columns.str.replace(' ', '_')
gossipcop_new_features.columns = gossipcop_new_features.columns.str.replace(' ', '_')

In [136]:
gossipcop_features = pd.concat([gossipcop_features, gossipcop_new_features], ignore_index=True)

In [137]:
gossipcop_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   news_id                    328 non-null    object 
 1   average_followers          328 non-null    float64
 2   average_follows            328 non-null    float64
 3   repost_total               328 non-null    int64  
 4   post_total                 328 non-null    int64  
 5   repost_percentage          328 non-null    float64
 6   label                      328 non-null    int64  
 7   average_repost             328 non-null    float64
 8   average_favorite           328 non-null    float64
 9   news_lifetime              328 non-null    float64
 10  nb_users_10_hours          328 non-null    int64  
 11  average_time_difference    328 non-null    float64
 12  retweet_percentage_1_hour  328 non-null    float64
dtypes: float64(8), int64(4), object(1)
memory usage: 3

In [139]:
gossipcop_features.to_csv("data/gossipcop_features.csv", index=False)

In [ ]:
news_propagation_fg = fs.get_feature_group(
    name="news_propagation",
    version=1,
)

In [94]:
historical_features = news_propagation_fg.select_all().read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.71s) 


In [140]:
historical_features = pd.concat([historical_features, gossipcop_new_features])

In [141]:
historical_features["news_lifetime"] = historical_features["news_lifetime"].astype(np.int64)

In [142]:
historical_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 487 entries, 0 to 87
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   news_id                    487 non-null    object 
 1   average_followers          487 non-null    float64
 2   average_follows            487 non-null    float64
 3   repost_total               487 non-null    int64  
 4   post_total                 487 non-null    int64  
 5   repost_percentage          487 non-null    float64
 6   average_repost             487 non-null    float64
 7   average_favorite           487 non-null    float64
 8   label                      487 non-null    int64  
 9   news_lifetime              487 non-null    int64  
 10  nb_users_10_hours          487 non-null    int64  
 11  average_time_difference    487 non-null    float64
 12  retweet_percentage_1_hour  487 non-null    float64
dtypes: float64(7), int64(5), object(1)
memory usage: 53.3+ K

In [143]:
historical_features.groupby("label").size()

label
0    277
1    210
dtype: int64

In [144]:
historical_features.columns

Index(['news_id', 'average_followers', 'average_follows', 'repost_total',
       'post_total', 'repost_percentage', 'average_repost', 'average_favorite',
       'label', 'news_lifetime', 'nb_users_10_hours',
       'average_time_difference', 'retweet_percentage_1_hour'],
      dtype='object')

In [145]:
historical_features.to_csv("data/historical_features.csv", index=False)

Save all the data into a Feature Store via Hopsworks

Insert dataframe into Feature Group, ```expectation_suite``` should contain data validation rules => we need to come up withd data validation rules

Do we actually need an event time ?

In [146]:
news_propagation_fg = fs.get_or_create_feature_group(
    name='news_propagation',
    description='Propagation characteristics of news in Bsky',
    version=1,
    primary_key=['news_id'],
    online_enabled=True,
)

In [147]:
news_propagation_fg.insert(historical_features, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 487/487 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: news_propagation_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1149079/jobs/named/news_propagation_1_offline_fg_materialization/executions
2025-01-07 16:26:35,115 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-07 16:26:38,308 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-07 16:28:04,689 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-01-07 16:28:04,850 INFO: Waiting for log aggregation to finish.
2025-01-07 16:28:33,627 INFO: Execution finished successfully.


(Job('news_propagation_1_offline_fg_materialization', 'SPARK'), None)

Add a description for each feature in the feature group.

In [ ]:
news_propagation_fg.update_feature_description("news_id", "The id of the news")
news_propagation_fg.update_feature_description("average_followers", "The average number of followers of the users who posted or reposted the news")
news_propagation_fg.update_feature_description("average_follows", "The average number of follows of the users who posted or reposted the news")
news_propagation_fg.update_feature_description("repost_total", "The total number of reposts of the news")
news_propagation_fg.update_feature_description("post_total", "The total number of posts of the news")
news_propagation_fg.update_feature_description("repost_percentage", "The percentage of reposts of the news out of the total number of posts and reposts")
news_propagation_fg.update_feature_description("average_repost", "The average number of reposts of the news")
news_propagation_fg.update_feature_description("average_favorite", "The average number of likes of the news")
news_propagation_fg.update_feature_description("label", "The label of the news : if it is fake or real")
news_propagation_fg.update_feature_description("news_lifetime", "The lifetime of the news, which is the time difference between the first and the last post (or repost) of the news")
news_propagation_fg.update_feature_description("nb_users_10_hours", "The number of users who posted (or reposted) the news in the first 10 hours")
news_propagation_fg.update_feature_description("average_time_difference", "The average time difference between a post and its reposts of the news")
news_propagation_fg.update_feature_description("retweet_percentage_1_hour", "The percentage of retweets of the news in the first hour")
